In [3]:
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os
from scipy import sparse
from patsy import dmatrix
from tqdm import tqdm

In [4]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr, pearsonr
from sklearn import ensemble

In [5]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [6]:
result = pd.read_csv('another_all_train_matrix.csv', sep=',')
result

,date,store_nbr,item_nbr,units,station_nbr,day,year,month,weekday,tmax,...,resultspeed,resultdir,avgspeed,date2,is_weekend,is_holiday,is_holiday_weekday,is_holiday_weekend,holiday_name,around_BlackFriday
0,2012-01-01,1,1,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
1,2012-01-01,1,2,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
2,2012-01-01,1,3,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
3,2012-01-01,1,4,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
4,2012-01-01,1,5,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
5,2012-01-01,1,6,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
6,2012-01-01,1,7,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
7,2012-01-01,1,8,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
8,2012-01-01,1,9,29,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else
9,2012-01-01,1,10,0,1,1,2012,1,6,0,...,0,0,0,2012-01-01,1,1,0,1,NewYearsDay,Else


In [7]:
result_2 = pd.read_csv('train_daydata_result_matrix.csv', sep=',')
result_2

,date,store_nbr,station_nbr,item_nbr,units,tmax,tmin,tavg,depart,dewpoint,...,SS,TS,UP,VA,VC,year,month,day,weekday,holiday_name
0,2012-01-01,1.0,1.0,9.0,29.0,52.0,31.0,42.0,16.0,36.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
1,2012-01-01,1.0,1.0,28.0,2.0,52.0,31.0,42.0,16.0,36.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
2,2012-01-01,1.0,1.0,51.0,1.0,52.0,31.0,42.0,16.0,36.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
3,2012-01-01,2.0,14.0,5.0,191.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
4,2012-01-01,2.0,14.0,44.0,215.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
5,2012-01-01,6.0,14.0,9.0,44.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
6,2012-01-01,6.0,14.0,45.0,86.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
7,2012-01-01,38.0,14.0,9.0,195.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
8,2012-01-01,38.0,14.0,45.0,135.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay
9,2012-01-01,42.0,14.0,5.0,154.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay


In [23]:
result_2.isnull().sum()

date                 0
store_nbr            0
station_nbr          0
item_nbr             0
units                0
tmax                 0
tmin                 0
tavg                 0
depart               0
dewpoint             0
wetbulb              0
heat                 0
cool                 0
sunrise              0
sunset               0
snowfall             0
preciptotal          0
stnpressure          0
sealevel             0
resultspeed          0
resultdir            0
avgspeed             0
+FC                  0
BC                   0
BL                   0
BR                   0
DR                   0
DS                   0
DU                   0
DZ                   0
                 ...  
FG+                  0
FU                   0
FZ                   0
GR                   0
GS                   0
HZ                   0
IC                   0
MI                   0
PL                   0
PO                   0
PR                   0
PY                   0
RA         

In [13]:
result_2['is_sunday'] = result_2['weekday'].isin(['Sun'])
result_2

,date,store_nbr,station_nbr,item_nbr,units,tmax,tmin,tavg,depart,dewpoint,...,TS,UP,VA,VC,year,month,day,weekday,holiday_name,is_sunday
0,2012-01-01,1.0,1.0,9.0,29.0,52.0,31.0,42.0,16.0,36.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
1,2012-01-01,1.0,1.0,28.0,2.0,52.0,31.0,42.0,16.0,36.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
2,2012-01-01,1.0,1.0,51.0,1.0,52.0,31.0,42.0,16.0,36.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
3,2012-01-01,2.0,14.0,5.0,191.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
4,2012-01-01,2.0,14.0,44.0,215.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
5,2012-01-01,6.0,14.0,9.0,44.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
6,2012-01-01,6.0,14.0,45.0,86.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
7,2012-01-01,38.0,14.0,9.0,195.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
8,2012-01-01,38.0,14.0,45.0,135.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True
9,2012-01-01,42.0,14.0,5.0,154.0,50.0,34.0,42.0,5.0,25.0,...,0.0,0.0,0.0,0.0,2012,1,1,Sun,NewYearsDay,True


In [22]:
model_1 = sm.OLS.from_formula("units ~ store_nbr + station_nbr + item_nbr + weekday + holiday_name + scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool) + scale(sunrise) + scale(snowfall) + scale(stnpressure) + scale(sealevel) + scale(resultspeed) + scale(resultdir) + scale(avgspeed) + BC + DZ + FG + FU + HZ + PL + RA + SH + SN + TS + VC", data=result_2)
print(model_1.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                  units   R-squared:                       0.124
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     15.00
Date:                Sat, 10 Mar 2018   Prob (F-statistic):          1.49e-109
Time:                        12:27:40   Log-Likelihood:                -25673.
No. Observations:                5009   AIC:                         5.144e+04
Df Residuals:                    4961   BIC:                         5.176e+04
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
In

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [8]:
ls_store_nbr = set(result_2.store_nbr)
mod = sys.modules[__name__]
df = pd.DataFrame()
for store_nbr in tqdm(ls_store_nbr):
    #setattr(mod, 'store_{}'.format(store_nbr), all_data_result[all_data_result['store_nbr'] == store_nbr])
    result_2 = result_2[result_2['store_nbr'] == store_nbr]

    df.append(result_2)
    setattr(mod, 'store_{}_data'.format(store_nbr), result_2)
    #mergedata.to_csv("store_{}_data.csv".format(store_nbr), sep=',', index=None)

100%|██████████| 45/45 [00:00<00:00, 69.98it/s]


In [12]:
store_1_data

NameError: name 'store_1_data' is not defined

In [10]:
model2 = sm.OLS.from_formula("units ~ C(weekday) + C(holiday_name) + is_holiday_weekend", data=store_1)
result_model2 = model2.fit()
print(result_model2.summary())

NameError: name 'store_1' is not defined